In [9]:
import pandas as pd
import numpy as np
import json, os, time, jieba


In [42]:
data_path = './data/AutoMaster_TrainSet.csv'
stop_words = './data/百度停用词表.txt'
save_path = './save/words_index.txt'
df = pd.read_csv(data_path)
stop_words = {el.strip() for el in open(stop_words, 'r', encoding='utf8').readlines() if el.strip()}

In [43]:
df['Dialogue'][0]

'技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了方向机带的有|车主说：[图片]|技师说：[语音]|车主说：有助力就是重，这车要匹配吧|技师说：不需要|技师说：你这是更换的部件有问题|车主说：跑快了还好点，就倒车重的很。|技师说：是非常重吗|车主说：是的，累人|技师说：[语音]|车主说：我觉得也是，可是车主是以前没这么重，选吧助理泵换了不行，又把放向机换了，现在还这样就不知道咋和车主解释。|技师说：[语音]|技师说：[语音]'

In [44]:
df

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手
5,Q6,雪佛兰,科鲁兹,科鲁兹变速箱旁边漏机油,技师说：具体是哪个部位呢？是发动机和变速器正中间位置吗？|车主说：[图片]|技师说：这个有拍...,像这种情况的话，如果机油不缺少，应该就是变速器的一轴油封有漏油的情况，需要拆下来检查一下，把...
6,Q7,丰田,凯美瑞(海外),我要怎么才能知道车子断开电瓶电源之后要不要做节气门或防盗，音响等的重置，还有节气门重置是不是...,技师说：你好！一般车子如果断开电瓶头着车启动后怠速偏低抖动就是节气门脏了需要重新清洗干净节气...,你好！一般车子如果断电后，重新着车启动会车子怠速偏低不稳抖动就是需要重新清洗干净节气门后匹配...
7,Q8,昌河,昌河Q35,昌河Q35音响怎么拆装,技师说：发个中控图片看看|车主说：[图片]|技师说：[图片]|技师说：圈起来的面板吗|车主说...,这个面板就是内部卡子固定，没有固定螺丝
8,Q9,长安,长安CS35,长安35朝阳轮胎不要里面的钢圈。用我自己的钢圈多少钱外面换多少钱,技师说：你的意思是就只更换一个新的轮胎就可以吧。|技师说：只更换单独的轮胎，这个价格是350...,这个单独购买轮胎，这个车胎的型号比较大在卖轮胎的地方，或者去修理厂他这个费用的话在350左右...
9,Q10,吉利汽车,远景,吉利远景外球笼上那个大螺丝是顺时针拧下来还是反的,技师说：逆时针旋转松|技师说：这个螺丝就是比较紧|车主说：扳手都断了|车主说：那就是说和轮胎...,这个螺丝跟轮胎螺丝一样的，顺丝抠


In [45]:
df = [df['Question'], df['Dialogue'], df['Report']]

In [46]:
df = [el.to_list() for el in df]

In [47]:
df = sum(df, [])

In [48]:
len(df)

248829

In [49]:
df = [el for el in df if type(el) == str]

In [50]:
df = [jieba.lcut(el) for el in df]

In [54]:
words = set()
for sentence in df:
    words.update(sentence)
words = words - stop_words

In [55]:
len(words)

106663

In [56]:
words = pd.DataFrame([[word, i] for i, word in enumerate(words)])

In [57]:
words.columns = ['index', 'word']

In [58]:
words

,index,word
0,八牛,0
1,挂扣,1
2,对换,2
3,2645,3
4,天逸前,4
5,库管,5
6,WINTER,6
7,路虎图,7
8,干是,8
9,圈线,9


In [59]:
words.to_csv(save_path, index=False)